# Logistics Data Cleaning Pipeline - Google Colab

## **Step 1: Setup, Load & Map Data**
**Description:**
We load the dataset and perform a critical "Column Mapping" step.
* **Fix 1:** We strip whitespace from raw column names to prevent `KeyError`.
* **Fix 2:** We rename the columns from your specific file (e.g., `shipping_mode`) to the standard names required for our analysis (e.g., `Shipment Mode`).
* **Fix 3:** We create placeholder columns for data that might be missing in the raw file (like `Season`) so the pipeline doesn't break.

In [36]:
import pandas as pd
import numpy as np

print("--- STEP 1: Loading & Mapping Real Data ---")

# 1. Load the file
filename = 'incom2024_delay_example_dataset.csv'
df = pd.read_csv(filename)

# 2. Clean Raw Headers (CRITICAL FIX)
# Removes hidden spaces like " shipping_mode " -> "shipping_mode"
df.columns = df.columns.str.strip()

# 3. Map your file's columns to the pipeline names
# This tells the code: "Use 'order_id' when I ask for 'Order ID'"
column_mapping = {
    'order_id': 'Order ID',
    'shipping_mode': 'Shipment Mode',
    'order_date': 'Scheduled Dispatch Date',   # Proxy: Order Date = Scheduled
    'shipping_date': 'Actual Dispatch Date',
    'order_city': 'Origin',
    'customer_city': 'Destination',
    'order_item_quantity': 'Package Weight',   # Proxy: Quantity = Weight
    'order_item_total_amount': 'Transport Cost', # Proxy: Amount = Cost
    'order_status': 'Delivery Priority'
}

# 4. Apply Renaming
df = df.rename(columns=column_mapping)

# 5. Handle "Transporter" column
# Your file doesn't have a 'Carrier' column, so we use Shipment Mode as a proxy
# or create a placeholder if you prefer.
if 'Transporter' not in df.columns:
    df['Transporter'] = df['Shipment Mode']

# 6. Create missing columns required for the pipeline
required_placeholders = ['Dispatch Delay (Days)', 'Delivery Time (Days)',
                         'Season', 'Holiday / Non-Holiday']

for col in required_placeholders:
    if col not in df.columns:
        df[col] = np.nan  # Initialize as empty, we will calculate/fill later

# 7. Verification
print(f"Dataset Loaded Successfully.")
print(f"Shape: {df.shape}")
print("Columns available:", df.columns.tolist())

# Check for the column that caused the error previously
if 'Shipment Mode' in df.columns:
    print("\n✅ SUCCESS: 'Shipment Mode' is ready.")
else:
    print("\n❌ ERROR: 'Shipment Mode' is missing.")

--- STEP 1: Loading & Mapping Real Data ---
Dataset Loaded Successfully.
Shape: (15549, 46)
Columns available: ['payment_type', 'profit_per_order', 'sales_per_customer', 'category_id', 'category_name', 'Destination', 'customer_country', 'customer_id', 'customer_segment', 'customer_state', 'customer_zipcode', 'department_id', 'department_name', 'latitude', 'longitude', 'market', 'Origin', 'order_country', 'order_customer_id', 'Scheduled Dispatch Date', 'Order ID', 'order_item_cardprod_id', 'order_item_discount', 'order_item_discount_rate', 'order_item_id', 'order_item_product_price', 'order_item_profit_ratio', 'Package Weight', 'sales', 'Transport Cost', 'order_profit_per_order', 'order_region', 'order_state', 'Delivery Priority', 'product_card_id', 'product_category_id', 'product_name', 'product_price', 'Actual Dispatch Date', 'Shipment Mode', 'label', 'Transporter', 'Dispatch Delay (Days)', 'Delivery Time (Days)', 'Season', 'Holiday / Non-Holiday']

✅ SUCCESS: 'Shipment Mode' is ready

## **Step 2: Detecting Data Issues**
**Description:**
We scan the now-standardized dataset for dirty data.
1.  **Missing Values:** Identifying where data is empty.
2.  **Duplicates:** Counting identical rows.
3.  **Logic Errors:** Checking for impossible negatives.

In [37]:
print("--- STEP 2: Detecting Data Issues ---")

# 1. Missing Values (Only showing columns that have nulls)
print("Missing Values Summary:")
missing = df.isnull().sum()
print(missing[missing > 0])

# 2. Duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicated Rows Detected: {duplicates}")

# 3. Logic Checks (Negative Values)
check_cols = ['Package Weight', 'Transport Cost', 'Delivery Time (Days)']
for col in check_cols:
    # Ensure column exists before checking
    if col in df.columns:
        # Count negatives
        neg_count = df[df[col] < 0].shape[0]
        if neg_count > 0:
            print(f"Warning: Found {neg_count} negative values in {col}")

--- STEP 2: Detecting Data Issues ---
Missing Values Summary:
Dispatch Delay (Days)    15549
Delivery Time (Days)     15549
Season                   15549
Holiday / Non-Holiday    15549
dtype: int64

Duplicated Rows Detected: 0


## **Step 3: Clean Dates & Validate Delays**
**Description:**
This step fixes the timeline data.
1.  Convert dates to standard format (`datetime`).
2.  **Calculate the Real Delay:** We subtract `Scheduled` from `Actual`.
3.  **Overwrite:** We replace the often inaccurate `Dispatch Delay` column with our calculated truth.

In [38]:
print("--- STEP 3: Cleaning Dates & Validating Logic (Fixed) ---")

# 1. Convert to datetime with utc=True to handle mixed timezones
# This fixes the "AttributeError" by ensuring we get a proper datetime64 dtype
df['Scheduled Dispatch Date'] = pd.to_datetime(df['Scheduled Dispatch Date'], errors='coerce', utc=True)
df['Actual Dispatch Date'] = pd.to_datetime(df['Actual Dispatch Date'], errors='coerce', utc=True)

# 2. Drop rows where we have no schedule (cannot measure delay without it)
df = df.dropna(subset=['Scheduled Dispatch Date'])

# 3. Calculate Delay (Actual - Scheduled)
# Now that both are proper datetimes, subtraction creates a Timedelta series
df['Delay_Calc'] = (df['Actual Dispatch Date'] - df['Scheduled Dispatch Date']).dt.days

# 4. Validation & Overwrite
# We trust our calculation over the raw data column
df['Dispatch Delay (Days)'] = df['Delay_Calc']

print("Dates converted (UTC) and delays validated/overwritten.")
# Check the types to confirm fix
print(df[['Scheduled Dispatch Date', 'Actual Dispatch Date']].dtypes)

--- STEP 3: Cleaning Dates & Validating Logic (Fixed) ---
Dates converted (UTC) and delays validated/overwritten.
Scheduled Dispatch Date    datetime64[ns, UTC]
Actual Dispatch Date       datetime64[ns, UTC]
dtype: object


# Step 4: Cleaning Categorical Variables
We standardize text columns to ensure reliable grouping and filtering.
1.  **Strip:** Remove leading/trailing spaces (e.g., " Air " → "Air").
2.  **Title Case:** Standardize capitalization (e.g., "AIR" → "Air").
3.  **Category Type:** Convert to pandas `category` dtype to save memory and speed up processing.

In [39]:
print("--- STEP 4: Cleaning Categorical Variables ---")

# List of categorical columns to clean
cat_cols = ['Shipment Mode', 'Transporter', 'Origin', 'Destination',
            'Season', 'Delivery Priority', 'Holiday / Non-Holiday']

for col in cat_cols:
    # Only process columns that actually exist in the dataframe
    if col in df.columns:
        # 1. Convert to string to handle any mixed types
        # 2. Strip whitespace
        # 3. Title case for consistency
        # 4. Convert to category
        df[col] = df[col].astype(str).str.strip().str.title().astype('category')

print("Categorical variables cleaned and standardized.")
# Verify one column to see the result
if 'Shipment Mode' in df.columns:
    print(f"Unique Shipment Modes: {df['Shipment Mode'].unique()}")

--- STEP 4: Cleaning Categorical Variables ---
Categorical variables cleaned and standardized.
Unique Shipment Modes: ['Standard Class', 'Second Class', 'First Class', 'Same Day']
Categories (4, object): ['First Class', 'Same Day', 'Second Class', 'Standard Class']


# Step 5: Cleaning Numeric Columns
We clean numerical data to handle errors and outliers:
1.  **Force Numeric:** Convert columns to numbers, turning errors (like "Five") into `NaN`.
2.  **Fix Negatives:** Convert negative weights or costs to positive using `abs()`.
3.  **Outlier Capping:** We cap values at the **1st and 99th percentiles**. This removes extreme data entry errors (e.g., a 5000kg package) without deleting rows, preserving the rest of the valid data.

In [40]:
print("--- STEP 5: Cleaning Numeric Columns ---")

num_cols = ['Package Weight', 'Dispatch Delay (Days)', 'Transport Cost', 'Delivery Time (Days)']

# 1. Force Numeric Types
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. Fix Negatives (Impossible values)
if 'Package Weight' in df.columns:
    df['Package Weight'] = df['Package Weight'].abs()
if 'Transport Cost' in df.columns:
    df['Transport Cost'] = df['Transport Cost'].abs()

# 3. Outlier Capping Function
def cap_outliers(series):
    # Only cap if there is enough data
    if series.count() > 10:
        lower = series.quantile(0.01)
        upper = series.quantile(0.99)
        return series.clip(lower=lower, upper=upper)
    return series

# 4. Apply Capping
print("Capping outliers at 1st and 99th percentiles...")
cols_to_cap = ['Package Weight', 'Transport Cost', 'Delivery Time (Days)']

for col in cols_to_cap:
    if col in df.columns:
        original_max = df[col].max()
        df[col] = cap_outliers(df[col])
        new_max = df[col].max()
        if original_max != new_max:
             print(f" -> Capped {col}: Max dropped from {original_max:.2f} to {new_max:.2f}")

--- STEP 5: Cleaning Numeric Columns ---
Capping outliers at 1st and 99th percentiles...
 -> Capped Transport Cost: Max dropped from 1939.99 to 453.36
 -> Capped Delivery Time (Days): Max dropped from nan to nan


# Step 6: Missing Value Treatment
We fill in missing data (`NaN`) using logic appropriate for each data type:
1.  **Numeric:** Impute with the **Median**. (Averages are unsafe in logistics due to skewed data).
2.  **Dates:** If `Actual Dispatch Date` is missing, we estimate it as: `Scheduled Date + Median Delay`.
3.  **Categorical:** Fill with "Unknown".

In [41]:

# 1. Numeric Imputation (Median)
for col in num_cols:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# 2. Date Imputation
# If Actual Date is missing, estimate it using Scheduled + Median Delay
median_delay = df['Dispatch Delay (Days)'].median()
# Ensure median_delay is not NaN; if it is, default to 0 for timedelta
if pd.isna(median_delay):
    median_delay = 0
df['Actual Dispatch Date'] = df['Actual Dispatch Date'].fillna(
    df['Scheduled Dispatch Date'] + pd.to_timedelta(median_delay, unit='D')
)

# 3. Categorical Imputation
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].cat.add_categories(['Unknown']).fillna('Unknown')

# Drop exact duplicates finally
df = df.drop_duplicates()
print("Missing values handled.")

Missing values handled.


# Step 7: Feature Engineering
We create new columns to support deeper business analysis:
1.  **Is_Delayed:** Binary flag (1 = Late, 0 = On Time/Early).
2.  **Delay_Category:** buckets delays into "Early", "On Time", "Slight", and "Serious".
3.  **Route:** Combines Origin and Destination.
4.  **Cost_per_kg:** Calculates cost efficiency.
5.  **Delivery_Efficiency:** Measures the gap between total time and delay.

In [42]:
print("--- STEP 7: Feature Engineering ---")

# 1. Binary Delay
if 'Dispatch Delay (Days)' in df.columns:
    df['Is_Delayed'] = np.where(df['Dispatch Delay (Days)'] > 0, 1, 0)

    # 2. Delay Categories
    # Bins: Anything < -0.01 is Early, -0.01 to 0 is On Time, etc.
    bins = [-float('inf'), -0.01, 0, 3, float('inf')]
    labels = ['Early', 'On Time', 'Slight Delay', 'Serious Delay']
    df['Delay_Category'] = pd.cut(df['Dispatch Delay (Days)'], bins=bins, labels=labels)

# 3. Route
if 'Origin' in df.columns and 'Destination' in df.columns:
    df['Route'] = df['Origin'].astype(str) + " -> " + df['Destination'].astype(str)

# 4. Cost Efficiency
if 'Transport Cost' in df.columns and 'Package Weight' in df.columns:
    # Avoid division by zero
    df['Cost_per_kg'] = df['Transport Cost'] / df['Package Weight'].replace(0, np.nan)

# 5. Delivery Efficiency
if 'Delivery Time (Days)' in df.columns and 'Dispatch Delay (Days)' in df.columns:
    df['Delivery_Efficiency'] = df['Delivery Time (Days)'] - df['Dispatch Delay (Days)']

print("New Features created successfully.")

--- STEP 7: Feature Engineering ---
New Features created successfully.


# Step 8: Final Health Check & Export
Finally, we verify the dataset is clean (no nulls, correct types) and export it to a CSV file for reporting or visualization.

In [43]:
print("--- STEP 8: Final Health Check & Export ---")

# Final check
print("\nFinal Null Counts:")
print(df.isnull().sum())

print("\nFinal Data Types:")
print(df.dtypes)

# Export
output_filename = "clean_incom2024_delay_dataset.csv"
df.to_csv(output_filename, index=False)
print(f"\nSuccessfully saved clean dataset to {output_filename}")

--- STEP 8: Final Health Check & Export ---

Final Null Counts:
payment_type                    0
profit_per_order                0
sales_per_customer              0
category_id                     0
category_name                   0
Destination                     0
customer_country                0
customer_id                     0
customer_segment                0
customer_state                  0
customer_zipcode                0
department_id                   0
department_name                 0
latitude                        0
longitude                       0
market                          0
Origin                          0
order_country                   0
order_customer_id               0
Scheduled Dispatch Date         0
Order ID                        0
order_item_cardprod_id          0
order_item_discount             0
order_item_discount_rate        0
order_item_id                   0
order_item_product_price        0
order_item_profit_ratio         0
Package Weight    